# Proyecto 7: Deploy de sistema de recomendación con Watson

En este proyecto llevaremos a cabo la puesta en producción del modelo entrenado en el proyecto 5. Es decir, lo subirmos la nube de IBM y utilizando llamados a la API de Watson tendremos acceso a él para realizar predicciones.

In [1]:
import warnings
warnings.filterwarnings("ignore")
import sklearn
from sklearn.datasets import load_files
moviedir = r'./dataset/movie_reviews' 
movie_reviews = load_files(moviedir, shuffle=True)

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 12)

In [3]:
import joblib
modelo = joblib.load('sentiment_36.pkl')

IBM Watson

**1) Cargá** la biblioteca `WatsonMachineLearningAPIClient`

In [4]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**2) Creá** variable con las credenciales que necesita `Watson`. Ellas son: `url, access_key, username, password e instance_id`

In [5]:
wml_credentials={
  "url": "https://us-south.ml.cloud.ibm.com",
  "apikey": "18bZqiG1mF34AJXbrMXZOCY4eXgR2PhMb8R_T2GnTLQ0",
  "instance_id": "8c95a5d8-655c-45c3-ba22-c8af0edcc6f4"
}

**3) Declará** la variable `client` y guardá en ella al objeto `WatsonMachineLearningAPIClient` con las credenciales como parámetro

In [6]:
client = WatsonMachineLearningAPIClient(wml_credentials)

**4) Creá** una variable que guarde las propiedades del modelo. Datos del autor y nombre del proyecto.

In [7]:
model_props = {
       client.repository.ModelMetaNames.NAME: 'Deploy de Sistema',
       client.repository.ModelMetaNames.DESCRIPTION: 'Acamica Entrega 7',
       client.repository.ModelMetaNames.AUTHOR_NAME: 'Ignacio Grosso',
       client.repository.ModelMetaNames.FRAMEWORK_NAME: 'sckikit-learn',
       client.repository.ModelMetaNames.FRAMEWORK_VERSION: '0.20.3'
}

**5) Hacé** un pipeline que contenga como primer paso a un `TfidfVectorizer` y como segundo paso, al mejor modelo que hayas obtenido en el proyecto 5. **Entrená** con este pipeline.

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 12)

In [9]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
pipeline = make_pipeline(vect, modelo)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth...001,
     verbose=0))],
         flatten_transform=None, n_jobs=None, voting='hard', weights=None))])

**6) Subí** al modelo a IBM Cloud usando `client.repository.store_model` con los parámetros correctos.

In [10]:
published_model = client.repository.store_model(model=pipeline, 
                                                meta_props=model_props, 
                                                training_data=X_train, 
                                                training_target=y_train)

**7) Obtené** el `uid` del modelo y guardalo en una variable.

In [11]:
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)

In [12]:
published_model_uid

'd4908c0f-e577-42ab-960e-3a8a5c2227fb'

In [14]:
models_details = client.repository.list_models()

------------------------------------  ----------------------  ------------------------  -----------------
GUID                                  NAME                    CREATED                   FRAMEWORK
d4908c0f-e577-42ab-960e-3a8a5c2227fb  Deploy de Sistema       2020-03-17T16:55:00.166Z  scikit-learn-0.20
23dc77c1-0819-466b-adf2-c3f7122b6580  Reviews classification  2020-03-17T16:40:51.015Z  scikit-learn-0.20
f9550b95-a239-42df-ae98-8f59b732f0e8  Reviews classification  2020-03-17T15:38:07.314Z  scikit-learn-0.20
44019c3b-76df-44c5-95f7-abdf398ce86a  Reviews classification  2020-03-17T15:30:13.585Z  scikit-learn-0.20
------------------------------------  ----------------------  ------------------------  -----------------


**8) Cargá** el modelo basado en su `uid` y utilizalo para realizar la predicción sobre el conjunto de test

In [15]:
loaded_model = client.repository.load(published_model_uid)
test_predictions = loaded_model.predict(X_test)

**9) Mostrar** el `classification_report` obtenido

In [16]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

print("roc_auc score: ", roc_auc_score(y_test, test_predictions))
print(classification_report(y_test, test_predictions))

roc_auc score:  0.8600000000000001
              precision    recall  f1-score   support

           0       0.82      0.92      0.87       200
           1       0.90      0.81      0.85       200

   micro avg       0.86      0.86      0.86       400
   macro avg       0.86      0.86      0.86       400
weighted avg       0.86      0.86      0.86       400

